In [4]:
%matplotlib notebook
#%matplotlib inline
import csv
#import serial
import time
import threading
import tkinter as tk
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import seaborn as sns
from math import atan2, asin, sqrt, degrees
import struct
import numpy as np
from pyquaternion import Quaternion
from scipy.spatial.transform import Rotation as Rt
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
import numpy as np

# def calculate_fha(quat1, quat2):
#     # Convert quaternions to rotation matrices
#     R1 = quaternion_to_matrix(quat1)
#     R2 = quaternion_to_matrix(quat2)

#     # Calculate the rotation matrix that takes us from quat1 to quat2
#     R = np.dot(R2, R1.T)

#     # Calculate the axis and angle of the rotation
#     axis, angle = matrix_to_axis_angle(R)

#     # Calculate the finite helical axis
#     FHA = axis * angle / (2*np.pi)

#     return FHA

def calculate_screw_params(quat1, quat2):
    
     # Convert quaternions to rotation matrices
    R1 = quaternion_to_matrix(quat1)
    R2 = quaternion_to_matrix(quat2)

    # Calculate the rotation matrix that takes us from quat1 to quat2
    R = np.dot(R2, R1.T)

    # Calculate the axis and angle of the rotation
    axis, angle = matrix_to_axis_angle(R)

    # Calculate the finite helical axis
    FHA = axis * angle / (2*np.pi)
    
    # Calculate the screw angle
    angle = np.linalg.norm(FHA)

    # Calculate the direction of the screw motion
    direction = FHA / angle

    # Calculate the interception point of the screw axis with the line of centers
    quat_center = slerp(quat1, quat2, 0.5)
    center = quaternion_to_vector(quat_center)
    fha_dot_center = np.dot(FHA, center)
    intercept = FHA * (fha_dot_center / np.dot(FHA, FHA))

    # Calculate the amount of translation along the screw axis
    t = np.dot(center - intercept, direction)

    return FHA, angle, direction, intercept, t

def quaternion_to_matrix(quat):
    # Convert quaternion to rotation matrix
    q0, q1, q2, q3 = quat
    R = np.array([[1-2*(q2**2+q3**2), 2*(q1*q2-q0*q3), 2*(q1*q3+q0*q2)],
                  [2*(q1*q2+q0*q3), 1-2*(q1**2+q3**2), 2*(q2*q3-q0*q1)],
                  [2*(q1*q3-q0*q2), 2*(q2*q3+q0*q1), 1-2*(q1**2+q2**2)]])
    return R

def matrix_to_axis_angle(R):
    # Calculate the axis and angle of the rotation matrix
    tr = np.trace(R)
    angle = np.arccos((tr-1)/2)
    if angle == 0:
        axis = np.array([1, 0, 0])
    else:
        axis = np.array([(R[2,1]-R[1,2])/(2*np.sin(angle)),
                         (R[0,2]-R[2,0])/(2*np.sin(angle)),
                         (R[1,0]-R[0,1])/(2*np.sin(angle))])
    return axis, angle

def quaternion_to_vector(quat):
    # Convert quaternion to 3D vector
    return np.array(quat[1:])

def slerp(quat1, quat2, t):
    # Spherical linear interpolation between two quaternions
    dot = np.dot(quat1, quat2)
    if dot < 0:
        quat1 = -quat1
        dot = -dot
    if dot > 0.9995:
        return (1-t)*quat1 + t*quat2
    angle = np.arccos(dot)
    return (np.sin((1-t)*angle)*quat1 + np.sin(t*angle)*quat2) / np.sin(angle)

# Example qu
    
def calculate_relative_quat(q1, q2):
    r1 = Rt.from_quat(q1)
    r2 = Rt.from_quat(q2)
    r = r2 * r1.inv()
    #print("r",r2)
    
    #T=np.array(len(q),np.identity(4))
    
    Rot=r.as_quat()
    #T[0:3, 0:3]=Rot

    return Rot
    # return r.as_euler('xyz', degrees=True)
    
def FiniteHelicalAxis(T):
    R = T[0:3, 0:3]
    v = T[0:3, 3]

    U = np.transpose(R) - R
#     print(U)

    n_orientation = 1 / (np.sqrt(((U[1, 2]) ** 2) + ((U[2, 0]) ** 2) + ((U[0, 1]) ** 2))) * (
    np.array((U[1, 2], U[2, 0], U[0, 1])))

    theta1 = np.arccos(0.5 * (R[0, 0] + R[1, 1] + R[2, 2] - 1)) * 180 / np.pi

    #   theta2[t] = np.arcsin(np.sqrt(U[1,2,t]**2 + U[2,0,t]**2 + U[0,1,t]**2))*180/np.pi

    translation = np.transpose(n_orientation).dot(v)

    c = ((1 + np.cos(theta1)) / (2 * (np.sin(theta1)) ** 2)) * (np.identity(3) - np.transpose(R)) @ v

    return n_orientation, theta1, translation, c
    
path2pts='/Users/nicolas/Desktop/3xflex-ext.csv'
data = np.genfromtxt(path2pts,skip_header=1,delimiter=',')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')



quaternion_timepts=[]


for timeframe in range (19,40):
    #print("timept", timeframe)
    
    q1 = [float(x) for x in data[timeframe,1:5]]
    q2 = [float(x) for x in data[timeframe,5:9]]

    rel_quaternion=calculate_relative_quat(q1, q2)
    
    # print(rel_quaternion)
    quaternion_timepts.append(rel_quaternion)

# print('quaternion_timepts', quaternion_timepts)
    
interval=1
for steps in range (0, 40, interval):
    
    q2a = Quaternion(a=quaternion_timepts[steps+interval][0], b=quaternion_timepts[steps+interval][1], c=quaternion_timepts[steps+interval][2], d=quaternion_timepts[steps+interval][3])
    q1a = Quaternion(a=quaternion_timepts[steps][0], b=quaternion_timepts[steps][1], c=quaternion_timepts[steps][2], d=quaternion_timepts[steps][3])

    mq2a=calculate_relative_quat(quaternion_timepts[steps], quaternion_timepts[steps+interval])
    quart1a = Quaternion(a=mq2a[0], b=mq2a[1], c=mq2a[2], d=mq2a[3])
    Rotation = quart1a.rotation_matrix         # 3x3 rotation matrix
    Transformation = quart1a.transformation_matrix # 4x4 rotation matrix
    
    quat1 = np.array([quaternion_timepts[steps][0], quaternion_timepts[steps][1], quaternion_timepts[steps][2], quaternion_timepts[steps][3]])
    quat2 = np.array([quaternion_timepts[steps+interval][0], quaternion_timepts[steps+interval][1], quaternion_timepts[steps+interval][2], quaternion_timepts[steps+interval][3]])

    n_orientation, theta1, translation, c=FiniteHelicalAxis(Transformation)
    fha, angle, direction, intercept, t=calculate_screw_params(quat1, quat2)
    # print(intercept)
    # print(intercept[0])
    

    
    # Plot FHA
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    # ax.plot([-1*fha[0], fha[0]], [-1*fha[1], fha[1]], [-1*fha[2], fha[2]],label=str(steps))
    ax.plot([intercept[0], fha[0]], [intercept[1], fha[1]], [intercept[2], fha[2]],label=str(steps))
    # ax.plot([intercept[0], direction[0]], [0, direction[1]], [0, direction[2]],label=str(steps))
    # ax.scatter(intercept)
plt.legend(loc = 'best')
plt.show()
    
   

    #print('n_orientation',n_orientation)
    #print('theta1',theta1)
    #print('translation',translation)
    #print('c',c)

    #print(quaternion_timepts[0])

In [ ]:

%matplotlib notebook
%matplotlib inline

import matplotlib.patches as mpatches


d=pd.read_csv('/Users/nicolas/Desktop/3xflex-ext.csv')
# print(d)
fig, axes = plt.subplots(1, 3, figsize=(10, 5))
sns.lineplot(ax=axes[0],data=d,x=d['Time'],y=d['x_diff'],color='blue',label='X')#6:16
axes[0].set_title('X - axis')


sns.lineplot(ax=axes[1],data=d,x=d['Time'],y=d['y_diff'],color='blue',label='Y')#6:16
axes[1].set_title('Y - axis')

sns.lineplot(ax=axes[2],data=d,x=d['Time'],y=d['z_diff'],color='blue',label='Z')#6:16
axes[2].set_title('Z - axis')

# plot_file_name="/Volumes/Luca_T5/4D_MSK/Dynamic_data/Jaw/Altered_motion.jpg"
# # plt.ylim((0,0.8))
 
# # save as jpeg
# plt.savefig(plot_file_name,
#                     format='jpeg',
#                     dpi=2000)

In [ ]:
# determine time intervals

step =10
path2pts='/Users/bkeelson/Downloads/3xflex-ext.csv'
data = np.genfromtxt(path2pts,skip_header=1,delimiter=',')
for i in range (0,len(data)):
    z_diff=data[i,-1]
    # previous=z_diff
    # print('previous', previous)
    
    if z_diff>0:
        q1 = [float(x) for x in data[timeframe,1:5]]
        q2 = [float(x) for x in data[timeframe,5:9]]

        rel_quaternion=calculate_relative_quat(q1, q2)
        current_zdiff=z_diff
        # print('current_zdiff', current_zdiff)
        # print('previous_zdiff', previous)
        interval =current_zdiff-previous
        previous=z_diff
        
        if interval>=step:
            print('interval', interval, 'is greater than or equal to step')
            print('current_zdiff', current_zdiff)
        
        
    else:
        # print ('Angle is negative moving on to the next angle')
        previous=z_diff
        
# timeframe=0
# print(data[timeframe,-1])